Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [11]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from heapq import heappush, heappop 

In [12]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

In [13]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

# Heuristic Function: Manhattan Distance

In [14]:
def manhattan_distance(state, goal):
    """Calculates the Manhattan distance of the state from the goal."""
    dist = 0
    for value in range(1, PUZZLE_DIM**2):
        x1, y1 = np.where(state == value)
        x2, y2 = np.where(goal == value)
        dist += abs(x1.item() - x2.item()) + abs(y1.item() - y2.item())
    return dist

# A* Search Implementation

In [15]:
def a_star_solver(start_state, goal_state):
    """Solves the n-puzzle using the A* search algorithm."""
    frontier = []
    heappush(frontier, (0, tuple(map(tuple, start_state)), []))  # Convert array to tuple for immutability
    visited = set()
    visited.add(tuple(map(tuple, start_state)))  # Use tuple representation for states

    total_nodes_evaluated = 0  # Track the number of nodes evaluated

    while frontier:
        total_nodes_evaluated += 1
        priority, current_state, path = heappop(frontier)
        
        # Convert current state back to numpy array
        current_state_array = np.array(current_state)

        # Check if the current state is the goal state
        if np.array_equal(current_state_array, goal_state):
            return path, total_nodes_evaluated
        
        # Expand the current state
        for action in available_actions(current_state_array):
            new_state_array = do_action(current_state_array, action)
            new_state = tuple(map(tuple, new_state_array))  # Convert new state to tuple
            if new_state not in visited:
                visited.add(new_state)
                cost = len(path) + 1  # g(n) = path length
                heuristic = manhattan_distance(new_state_array, goal_state)  # h(n)
                heappush(frontier, (cost + heuristic, new_state, path + [action]))
    return None, total_nodes_evaluated  # No solution found

# Create the goal state

In [16]:
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

# Randomize the puzzle

In [17]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[ 5,  3,  1, 12],
       [ 2,  0, 14,  7],
       [ 4, 11,  6, 15],
       [13, 10,  9,  8]])

# Display the Initial and Goal States

In [18]:
print("Initial State:")
print(state)
print("\nGoal State:")
print(goal_state)

Initial State:
[[ 5  3  1 12]
 [ 2  0 14  7]
 [ 4 11  6 15]
 [13 10  9  8]]

Goal State:
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]]


# Solve the Puzzle

In [19]:
solution, total_nodes_evaluated = a_star_solver(state, goal_state)

# Display Results

In [21]:
if solution:
    quality = len(solution)  # Number of actions in the solution
    cost = total_nodes_evaluated  # Total number of nodes evaluated
    efficiency = quality / cost  # Efficiency: quality vs cost
    
    print(f"\nSolution Found!")
    print(f"Quality: {quality} (Number of actions in the solution)")
    print(f"Cost: {cost} (Total number of actions evaluated)")
    print(f"Efficiency: {efficiency:.4f} (Quality vs Cost)")
else:
    print("\nNo solution found.")


Solution Found!
Quality: 48 (Number of actions in the solution)
Cost: 6653987 (Total number of actions evaluated)
Efficiency: 0.0000 (Quality vs Cost)
